In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-rc0


In [2]:
def make_toy_dataset(num_data):
    X = np.random.randn(num_data, 3)
    y = 3 * X[:, 0] - 2 * X[:, 1]**3 + 2 * X[:, 2]**2 + 0.5 * np.random.randn(num_data)
    y = y[:, np.newaxis]
    return X, y

In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Dense(10, activation="relu", input_dim=3),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ])

    return model

In [4]:
N_train = 40000
N_test = 4000

num_epochs = 10
learning_rate = 0.01
batch_size = 64

In [5]:
# get toy datasets
X_train, y_train = make_toy_dataset(N_train)
X_test, y_test = make_toy_dataset(N_test)

# make datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=512).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100)

In [6]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

model.compile(optimizer, loss="mse", metrics=["mae"])

start = time.time()
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(X_test, y_test))
end = time.time()

print(end - start, "(sec)")

Train on 40000 samples, validate on 4000 samples
Epoch 1/10
40000/40000 [==============================] - 2s 60us/sample - loss: 9.3465 - mae: 1.6021 - val_loss: 3.0168 - val_mae: 0.9789
Epoch 2/10
40000/40000 [==============================] - 1s 35us/sample - loss: 2.5969 - mae: 1.0369 - val_loss: 2.8398 - val_mae: 1.0157
Epoch 3/10
40000/40000 [==============================] - 1s 33us/sample - loss: 2.1557 - mae: 0.9323 - val_loss: 2.3046 - val_mae: 0.8977
Epoch 4/10
40000/40000 [==============================] - 1s 35us/sample - loss: 1.6998 - mae: 0.8437 - val_loss: 1.7466 - val_mae: 0.6958
Epoch 5/10
40000/40000 [==============================] - 1s 35us/sample - loss: 1.4499 - mae: 0.7754 - val_loss: 1.5472 - val_mae: 0.7006
Epoch 6/10
40000/40000 [==============================] - 1s 35us/sample - loss: 1.2842 - mae: 0.7407 - val_loss: 1.3635 - val_mae: 0.6287
Epoch 7/10
40000/40000 [==============================] - 1s 35us/sample - loss: 1.1056 - mae: 0.7160 - val_loss: 1.3

In [7]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()

start = time.time()

for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))

end = time.time()
print(end - start, "(sec)")

W0827 19:49:31.786652 11664 base_layer.py:1772] Layer dense_3 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 1: loss=7.460, val_loss=2.745130, mae=1.415, val_mae=0.968
epoch 2: loss=2.092, val_loss=1.492179, mae=0.850, val_mae=0.716
epoch 3: loss=1.300, val_loss=1.134630, mae=0.714, val_mae=0.641
epoch 4: loss=1.130, val_loss=1.307690, mae=0.655, val_mae=0.718
epoch 5: loss=0.907, val_loss=0.884154, mae=0.632, val_mae=0.583
epoch 6: loss=1.239, val_loss=0.946404, mae=0.654, val_mae=0.591
epoch 7: loss=0.888, val_loss=1.181401, mae=0.617, val_mae=0.639
epoch 8: loss=0.714, val_loss=0.867503, mae=0.589, val_mae=0.576
epoch 9: loss=0.718, val_loss=0.833562, mae=0.579, val_mae=0.552
epoch 10: loss=0.765, val_loss=0.917903, mae=0.573, val_mae=0.558
72.75276517868042 (sec)


In [8]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

W0827 19:50:44.817705 11664 base_layer.py:1772] Layer dense_6 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 1: loss=8.643, val_loss=4.620719, mae=1.442, val_mae=1.135
epoch 2: loss=2.975, val_loss=2.078732, mae=0.994, val_mae=0.850
epoch 3: loss=2.513, val_loss=1.819898, mae=0.918, val_mae=0.799
epoch 4: loss=1.661, val_loss=1.325308, mae=0.825, val_mae=0.719
epoch 5: loss=1.781, val_loss=1.311718, mae=0.796, val_mae=0.678
epoch 6: loss=1.443, val_loss=1.115543, mae=0.751, val_mae=0.639
epoch 7: loss=1.146, val_loss=1.039012, mae=0.706, val_mae=0.628
epoch 8: loss=0.952, val_loss=0.927890, mae=0.667, val_mae=0.598
epoch 9: loss=1.134, val_loss=1.366605, mae=0.683, val_mae=0.708
epoch 10: loss=0.882, val_loss=1.005251, mae=0.657, val_mae=0.578
29.26610279083252 (sec)


In [9]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def train_epoch(train_dataset):
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)


@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
@tf.function
def eval_epoch(test_dataset):
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)

    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    train_epoch(train_dataset)
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    eval_epoch(test_dataset)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

W0827 19:51:14.374773 11664 base_layer.py:1772] Layer dense_9 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 1: loss=10.335, val_loss=2.975882, mae=1.653, val_mae=0.912
epoch 2: loss=2.382, val_loss=2.124567, mae=0.930, val_mae=0.846
epoch 3: loss=1.757, val_loss=2.103338, mae=0.839, val_mae=0.866
epoch 4: loss=1.866, val_loss=1.661618, mae=0.833, val_mae=0.767
epoch 5: loss=1.204, val_loss=1.271695, mae=0.735, val_mae=0.679
epoch 6: loss=1.003, val_loss=1.104320, mae=0.691, val_mae=0.612
epoch 7: loss=0.977, val_loss=1.415838, mae=0.676, val_mae=0.727
epoch 8: loss=1.225, val_loss=1.228926, mae=0.710, val_mae=0.664
epoch 9: loss=0.884, val_loss=1.038066, mae=0.649, val_mae=0.617
epoch 10: loss=0.879, val_loss=1.208016, mae=0.642, val_mae=0.735
8.183550357818604 (sec)


In [10]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def train_epoch(train_dataset):
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)



def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
@tf.function
def eval_epoch(test_dataset):
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)

    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    train_epoch(train_dataset)
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    eval_epoch(test_dataset)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

W0827 19:51:22.686779 11664 base_layer.py:1772] Layer dense_12 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 1: loss=8.643, val_loss=2.838111, mae=1.531, val_mae=0.925
epoch 2: loss=2.292, val_loss=2.249154, mae=0.927, val_mae=0.783
epoch 3: loss=1.619, val_loss=1.557780, mae=0.810, val_mae=0.692
epoch 4: loss=1.399, val_loss=1.262913, mae=0.762, val_mae=0.674
epoch 5: loss=1.201, val_loss=1.769752, mae=0.718, val_mae=0.916
epoch 6: loss=1.505, val_loss=1.322963, mae=0.776, val_mae=0.681
epoch 7: loss=1.117, val_loss=1.249477, mae=0.714, val_mae=0.670
epoch 8: loss=1.120, val_loss=1.240016, mae=0.708, val_mae=0.684
epoch 9: loss=0.906, val_loss=1.086592, mae=0.671, val_mae=0.693
epoch 10: loss=1.065, val_loss=1.078038, mae=0.679, val_mae=0.623
6.897313117980957 (sec)
